In [2]:
from pymongo import MongoClient
import pandas as pd

## Assemble data and mount dataframe

First we need to get all the data from mongodb and mount in a dataframe so its easier to work with

In [3]:
# Creating connection to the DB
conn = MongoClient('localhost', 27017)
db = conn.nbascrapper
players_collection = db.players
players_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nbascrapper'), 'players')

In [4]:
dfs = []
# Unpack json array to df
def json_to_df(row, json_col):
    try:
        json_df = pd.DataFrame(row[json_col])
        dfs.append(json_df.assign(**row.drop(json_col)))
    except:
        print(row[json_col])

In [109]:
# Getting all 'Career Regular Season Stats' from players
query_filter = { 
        "stats": 
        {
            "$not":
            {
                "$eq":{}
            }
        },
        "$or":[
            {
                "stats.Career Regular Season Stats": 
                {
                        "$exists": True
                }
            },{
                "stats.Traditional Splits": 
                {
                        "$exists": True
                }
            },
        ]
    }

query_project = {
    '_id': 0,
    'name': 1,
    'position': 1,
    'stats.Career Regular Season Stats': 1,
    'stats.Traditional Splits': 1
}

data = list(players_collection.find(query_filter, query_project))

df_unpacked = pd.io.json.json_normalize(data)
df_unpacked['stats'] = df_unpacked['stats.Career Regular Season Stats'].combine_first(df_unpacked['stats.Traditional Splits'])
del df_unpacked['stats.Career Regular Season Stats']
del df_unpacked['stats.Traditional Splits']
df_unpacked.apply(json_to_df, axis=1, json_col='stats')

players = pd.concat(dfs)
del 

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/media/lucas/DATA/Repos/nba_scrapper/venv/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## Analysing dataframe

In [110]:
print(f'Nº Players: {len(players)}')
print(f'Nº Columns: {len(list(players))}')
print(f'Columns: {list(players)}')
players.drop_duplicates(subset=['name', 'TEAM','Season'], keep='first')
players.head()

Nº Players: 29241
Nº Columns: 88
Columns: ['%3PA', '%3PM', '%AST', '%BLK', '%BLKA', '%DREB', '%FGA', '%FGA  2PT', '%FGA  3PT', '%FGM', '%FTA', '%FTM', '%OREB', '%PF', '%PFD', '%PTS', '%PTS  2PT', '%PTS  2PT\xa0MR', '%PTS  3PT', '%PTS  FBPs', '%PTS  FT', '%PTS  OffTO', '%PTS  PITP', '%REB', '%STL', '%TOV', '+/-', '2FGM  %AST', '2FGM  %UAST', '2nd\xa0PTS', '3FGM  %AST', '3FGM  %UAST', '3P%', '3PA', '3PM', 'AGE', 'AST', 'AST Ratio', 'AST%', 'AST/TO', 'BLK', 'BLKA', 'By Year', 'DD2', 'DREB', 'DREB%', 'DefRtg', 'FBPs', 'FG%', 'FGA', 'FGM', 'FGM  %AST', 'FGM  %UAST', 'FP', 'FT%', 'FTA', 'FTM', 'GP', 'GS', 'MIN', 'NetRtg', 'OREB', 'OREB%', 'OffRtg', 'Opp  2nd\xa0PTS', 'Opp  FBPs', 'Opp  PITP', 'Opp  PTS\xa0OFF\xa0TO', 'PACE', 'PF', 'PFD', 'PIE', 'PITP', 'PTS', 'PTS\xa0OFF\xa0TO', 'REB', 'REB%', 'STL', 'Season', 'TD3', 'TEAM', 'TO Ratio', 'TOV', 'TS%', 'USG%', 'eFG%', 'name', 'position']


,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGA 2PT,%FGA 3PT,%FGM,...,Season,TD3,TEAM,TO Ratio,TOV,TS%,USG%,eFG%,name,position
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-95,NaN,SAC,NaN,0.8,NaN,NaN,NaN,Alaa Abdelnaby,F
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-95,NaN,PHL,NaN,1.7,NaN,NaN,NaN,Alaa Abdelnaby,F
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-95,NaN,TOT,NaN,0.8,NaN,NaN,NaN,Alaa Abdelnaby,F
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1993-94,NaN,BOS,NaN,1.3,NaN,NaN,NaN,Alaa Abdelnaby,F
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1992-93,NaN,MIL,NaN,1.1,NaN,NaN,NaN,Alaa Abdelnaby,F


In [111]:
# Fixing column names
players.columns = [col.replace('\xa0', '').replace('  ', '').replace(' ', '') for col in players.columns]
print(f'Columns: {list(players)}')

Columns: ['%3PA', '%3PM', '%AST', '%BLK', '%BLKA', '%DREB', '%FGA', '%FGA2PT', '%FGA3PT', '%FGM', '%FTA', '%FTM', '%OREB', '%PF', '%PFD', '%PTS', '%PTS2PT', '%PTS2PTMR', '%PTS3PT', '%PTSFBPs', '%PTSFT', '%PTSOffTO', '%PTSPITP', '%REB', '%STL', '%TOV', '+/-', '2FGM%AST', '2FGM%UAST', '2ndPTS', '3FGM%AST', '3FGM%UAST', '3P%', '3PA', '3PM', 'AGE', 'AST', 'ASTRatio', 'AST%', 'AST/TO', 'BLK', 'BLKA', 'ByYear', 'DD2', 'DREB', 'DREB%', 'DefRtg', 'FBPs', 'FG%', 'FGA', 'FGM', 'FGM%AST', 'FGM%UAST', 'FP', 'FT%', 'FTA', 'FTM', 'GP', 'GS', 'MIN', 'NetRtg', 'OREB', 'OREB%', 'OffRtg', 'Opp2ndPTS', 'OppFBPs', 'OppPITP', 'OppPTSOFFTO', 'PACE', 'PF', 'PFD', 'PIE', 'PITP', 'PTS', 'PTSOFFTO', 'REB', 'REB%', 'STL', 'Season', 'TD3', 'TEAM', 'TORatio', 'TOV', 'TS%', 'USG%', 'eFG%', 'name', 'position']


#### Columns description (As described in https://stats.nba.com/help/glossary/)

In [112]:
# Print all text in column
pd.options.display.max_colwidth = 200

query_project = {
    '_id': 0
}

glossary_collection = db.glossary
glossary = pd.DataFrame(list(glossary_collection.find({},query_project)))
glossary['Key'] = glossary['Key'].str.upper()
glossary['Key'] = glossary['Key'].str.replace(' ', '')
glossary.set_index('Key', inplace=True)
glossary.fillna('', inplace=True)

for column in list(players):
    try:
        name = glossary.loc[column.upper()]["Definition"]
    except KeyError:
        name = "Not in list!"
    
    print('|----------------------------------------------------------------------------------------------------------------|')
    print(f'|{column}: {name}')
    print('|----------------------------------------------------------------------------------------------------------------|')

|----------------------------------------------------------------------------------------------------------------|
|%3PA: The percentage of a team's 3 point field goals attempted that a player has while on the court  
|----------------------------------------------------------------------------------------------------------------|
|----------------------------------------------------------------------------------------------------------------|
|%3PM: The percentage of a team's 3 point field goals that a player has made while on the court  
|----------------------------------------------------------------------------------------------------------------|
|----------------------------------------------------------------------------------------------------------------|
|%AST: The percentage of a team's assists that a player has while on the court  
|----------------------------------------------------------------------------------------------------------------|
|---------------------------

In [113]:
print(players[['Season', 'ByYear', 'GS']])
players[['Season', 'ByYear', 'GS']].describe()

      Season   ByYear    GS
0    1994-95      NaN     0
1    1994-95      NaN     0
2    1994-95      NaN     0
3    1993-94      NaN     0
4    1992-93      NaN     0
5    1992-93      NaN    52
6    1992-93      NaN    52
7    1991-92      NaN     1
8    1990-91      NaN     0
9   Overall:      NaN    53
0    1977-78      NaN   NaN
1    1977-78      NaN   NaN
2    1977-78      NaN   NaN
3    1976-77      NaN   NaN
4    1975-76      NaN   NaN
5    1974-75      NaN   NaN
6    1973-74      NaN   NaN
7    1972-73      NaN   NaN
8    1971-72      NaN   NaN
9    1970-71      NaN   NaN
10   1969-70      NaN   NaN
11   1968-69      NaN   NaN
12   1968-69      NaN   NaN
13   1968-69      NaN   NaN
14  Overall:      NaN   NaN
0    1988-89      NaN    74
1    1987-88      NaN    80
2    1986-87      NaN    78
3    1985-86      NaN    79
4    1984-85      NaN    79
5    1983-84      NaN    80
6    1982-83      NaN    79
7    1981-82      NaN    76
8    1980-81      NaN    80
9    1979-80      Na

,Season,ByYear,GS
count,18427,10814,10595
unique,71,24,462
top,Overall:,2018-19,0
freq,2648,696,2782


In [114]:
players.describe()

,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGA2PT,%FGA3PT,%FGM,...,PFD,PIE,PITP,PTSOFFTO,REB%,TD3,TORatio,TS%,USG%,eFG%
count,29.000000,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000,1216.000000,1216.000000,29.000000,...,2235.000000,222.000000,2235.000000,2235.000000,222.000000,7112.000000,222.000000,222.000000,251.000000,222.000000
mean,19.127586,18.306897,20.893103,16.831034,19.57931,19.365517,17.765517,77.114967,22.309457,17.310345,...,1.416421,9.950450,3.445369,1.384430,8.931081,0.075506,10.657658,54.137838,20.434661,50.712162
std,17.920856,21.291412,13.458718,13.305104,9.15140,7.150638,6.833075,22.996852,22.299697,6.088827,...,1.329873,5.249794,2.642465,1.044896,4.319133,0.909369,3.561053,8.675597,6.954914,9.402810
min,0.000000,0.000000,3.700000,0.000000,0.00000,7.800000,6.700000,0.000000,0.000000,6.700000,...,0.000000,-25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.000000,10.700000,7.200000,14.30000,14.300000,13.400000,60.000000,0.800000,15.000000,...,0.400000,7.300000,1.400000,0.600000,5.900000,0.000000,8.500000,51.525000,15.200000,47.625000
50%,21.300000,21.100000,18.400000,15.000000,19.50000,18.400000,16.100000,81.100000,17.900000,16.700000,...,1.100000,9.150000,2.800000,1.200000,7.900000,0.000000,10.300000,54.500000,18.500000,50.550000
75%,26.700000,27.300000,28.300000,21.100000,25.00000,26.400000,19.800000,99.200000,39.100000,18.400000,...,2.000000,12.700000,4.800000,1.900000,10.475000,0.000000,12.250000,58.200000,26.550000,54.100000
max,75.000000,100.000000,48.500000,50.000000,36.70000,34.800000,42.900000,100.000000,100.000000,37.500000,...,9.500000,22.600000,14.100000,7.100000,23.600000,42.000000,25.000000,105.000000,38.900000,100.000000


## Cleaning dataframe

In [115]:
players2 = players.copy()
players2['Season'] = players2['Season'].combine_first(players2['ByYear'])
players2[players2['name'].str.contains('Bob Pettit')][['Season','name','TEAM']].sort_values('Season').head()

,Season,name,TEAM
10,1954-55,Bob Pettit,MIH
9,1955-56,Bob Pettit,STL
8,1956-57,Bob Pettit,STL
7,1957-58,Bob Pettit,STL
6,1958-59,Bob Pettit,STL


In [116]:
# Removing useless columns
del players2['GS']
del players2['ByYear']
del players2['%FGA2PT']
del players2['%FGA3PT']
del players2['2FGM%AST']
del players2['2FGM%UAST']
del players2['3FGM%AST']
del players2['3FGM%UAST']
del players2['FGM%AST']
del players2['FGM%UAST']

In [117]:
players2.describe()

,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGM,%FTA,%FTM,...,PFD,PIE,PITP,PTSOFFTO,REB%,TD3,TORatio,TS%,USG%,eFG%
count,29.000000,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000,29.000000,29.000000,29.000000,...,2235.000000,222.000000,2235.000000,2235.000000,222.000000,7112.000000,222.000000,222.000000,251.000000,222.000000
mean,19.127586,18.306897,20.893103,16.831034,19.57931,19.365517,17.765517,17.310345,17.227586,15.186207,...,1.416421,9.950450,3.445369,1.384430,8.931081,0.075506,10.657658,54.137838,20.434661,50.712162
std,17.920856,21.291412,13.458718,13.305104,9.15140,7.150638,6.833075,6.088827,9.363451,11.114975,...,1.329873,5.249794,2.642465,1.044896,4.319133,0.909369,3.561053,8.675597,6.954914,9.402810
min,0.000000,0.000000,3.700000,0.000000,0.00000,7.800000,6.700000,6.700000,0.000000,0.000000,...,0.000000,-25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.000000,10.700000,7.200000,14.30000,14.300000,13.400000,15.000000,12.300000,10.300000,...,0.400000,7.300000,1.400000,0.600000,5.900000,0.000000,8.500000,51.525000,15.200000,47.625000
50%,21.300000,21.100000,18.400000,15.000000,19.50000,18.400000,16.100000,16.700000,15.800000,14.100000,...,1.100000,9.150000,2.800000,1.200000,7.900000,0.000000,10.300000,54.500000,18.500000,50.550000
75%,26.700000,27.300000,28.300000,21.100000,25.00000,26.400000,19.800000,18.400000,20.000000,16.800000,...,2.000000,12.700000,4.800000,1.900000,10.475000,0.000000,12.250000,58.200000,26.550000,54.100000
max,75.000000,100.000000,48.500000,50.000000,36.70000,34.800000,42.900000,37.500000,47.900000,51.900000,...,9.500000,22.600000,14.100000,7.100000,23.600000,42.000000,25.000000,105.000000,38.900000,100.000000


In [118]:
del players2['%PTS2PT']
del players2['%PTS3PT']
del players2['%PTSFBPs']
del players2['%PTSFT']
del players2['%PTSOffTO']
del players2['%PTSPITP']

In [119]:
players2.describe()

,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGM,%FTA,%FTM,...,PFD,PIE,PITP,PTSOFFTO,REB%,TD3,TORatio,TS%,USG%,eFG%
count,29.000000,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000,29.000000,29.000000,29.000000,...,2235.000000,222.000000,2235.000000,2235.000000,222.000000,7112.000000,222.000000,222.000000,251.000000,222.000000
mean,19.127586,18.306897,20.893103,16.831034,19.57931,19.365517,17.765517,17.310345,17.227586,15.186207,...,1.416421,9.950450,3.445369,1.384430,8.931081,0.075506,10.657658,54.137838,20.434661,50.712162
std,17.920856,21.291412,13.458718,13.305104,9.15140,7.150638,6.833075,6.088827,9.363451,11.114975,...,1.329873,5.249794,2.642465,1.044896,4.319133,0.909369,3.561053,8.675597,6.954914,9.402810
min,0.000000,0.000000,3.700000,0.000000,0.00000,7.800000,6.700000,6.700000,0.000000,0.000000,...,0.000000,-25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.000000,10.700000,7.200000,14.30000,14.300000,13.400000,15.000000,12.300000,10.300000,...,0.400000,7.300000,1.400000,0.600000,5.900000,0.000000,8.500000,51.525000,15.200000,47.625000
50%,21.300000,21.100000,18.400000,15.000000,19.50000,18.400000,16.100000,16.700000,15.800000,14.100000,...,1.100000,9.150000,2.800000,1.200000,7.900000,0.000000,10.300000,54.500000,18.500000,50.550000
75%,26.700000,27.300000,28.300000,21.100000,25.00000,26.400000,19.800000,18.400000,20.000000,16.800000,...,2.000000,12.700000,4.800000,1.900000,10.475000,0.000000,12.250000,58.200000,26.550000,54.100000
max,75.000000,100.000000,48.500000,50.000000,36.70000,34.800000,42.900000,37.500000,47.900000,51.900000,...,9.500000,22.600000,14.100000,7.100000,23.600000,42.000000,25.000000,105.000000,38.900000,100.000000


In [120]:
# Remove useless rows
players2 = players2[~players2['Season'].str.contains('Overall', na=False)]
players2.head()

,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGM,%FTA,%FTM,...,Season,TD3,TEAM,TORatio,TOV,TS%,USG%,eFG%,name,position
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-95,NaN,SAC,NaN,0.8,NaN,NaN,NaN,Alaa Abdelnaby,F
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-95,NaN,PHL,NaN,1.7,NaN,NaN,NaN,Alaa Abdelnaby,F
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1994-95,NaN,TOT,NaN,0.8,NaN,NaN,NaN,Alaa Abdelnaby,F
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1993-94,NaN,BOS,NaN,1.3,NaN,NaN,NaN,Alaa Abdelnaby,F
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1992-93,NaN,MIL,NaN,1.1,NaN,NaN,NaN,Alaa Abdelnaby,F


## Joining with result dataframe

In [121]:
awards_collection = db.awards
dfs = []
# Getting all 'MVPS' from awards
query_filter = {}

query_project = {
    '_id': 0,
    'MVP': 1
}

data = list(awards_collection.find(query_filter, query_project))

df_unpacked = pd.io.json.json_normalize(data)
df_unpacked.apply(json_to_df, axis=1, json_col='MVP')

mvps = pd.concat(dfs)
print(len(mvps))
mvps

64


,YEAR,PLAYER,TEAM
0,2018-19,Giannis Antetokounmpo,Milwaukee Bucks
1,2017-18,James Harden,Houston Rockets
2,2016-17,Russell Westbrook,Oklahoma City Thunder
3,2015-16,Stephen Curry,Golden State Warriors
4,2014-15,Stephen Curry,Golden State Warriors
5,2013-14,Kevin Durant,Oklahoma City Thunder
6,2012-13,LeBron James,Miami Heat
7,2011-12,LeBron James,Miami Heat
8,2010-11,Derrick Rose,Chicago Bulls
9,2009-10,LeBron James,Cleveland Cavaliers


#### Normalizing team name

In [122]:
teams_collection = db.teams
dfs = []
# Getting all 'teams'
query_filter = {}

query_project = {
    '_id': 0
}

data = list(teams_collection.find(query_filter, query_project))

teams = pd.DataFrame(data)
teams = teams.rename(columns={'NAME':'TEAM'})
teams.head()

,KEY,TEAM
0,ATL,Atlanta Hawks
1,BKN,Brooklyn Nets
2,BOS,Boston Celtics
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls


In [123]:
mvps2 = pd.merge(mvps, teams, on='TEAM')
print(len(mvps2))
mvps2

64


,YEAR,PLAYER,TEAM,KEY
0,2018-19,Giannis Antetokounmpo,Milwaukee Bucks,MIL
1,1973-74,Kareem Abdul-Jabbar,Milwaukee Bucks,MIL
2,1971-72,Kareem Abdul-Jabbar,Milwaukee Bucks,MIL
3,1970-71,Kareem Abdul-Jabbar,Milwaukee Bucks,MIL
4,2017-18,James Harden,Houston Rockets,HOU
5,1993-94,Hakeem Olajuwon,Houston Rockets,HOU
6,1981-82,Moses Malone,Houston Rockets,HOU
7,1978-79,Moses Malone,Houston Rockets,HOU
8,2016-17,Russell Westbrook,Oklahoma City Thunder,OKC
9,2013-14,Kevin Durant,Oklahoma City Thunder,OKC


In [124]:
pd.set_option('display.max_rows', None)
mvps2['TEAM'] = mvps2['KEY']
mvps2['MVP'] = 1
mvps2 = mvps2.rename(columns={'PLAYER':'name','YEAR': 'Season'})
del mvps2['KEY']

print(mvps2)

     Season                   name TEAM  MVP
0   2018-19  Giannis Antetokounmpo  MIL    1
1   1973-74    Kareem Abdul-Jabbar  MIL    1
2   1971-72    Kareem Abdul-Jabbar  MIL    1
3   1970-71    Kareem Abdul-Jabbar  MIL    1
4   2017-18           James Harden  HOU    1
5   1993-94        Hakeem Olajuwon  HOU    1
6   1981-82           Moses Malone  HOU    1
7   1978-79           Moses Malone  HOU    1
8   2016-17      Russell Westbrook  OKC    1
9   2013-14           Kevin Durant  OKC    1
10  2015-16          Stephen Curry  GSW    1
11  2014-15          Stephen Curry  GSW    1
12  2012-13           LeBron James  MIA    1
13  2011-12           LeBron James  MIA    1
14  2010-11           Derrick Rose  CHI    1
15  1997-98         Michael Jordan  CHI    1
16  1995-96         Michael Jordan  CHI    1
17  1991-92         Michael Jordan  CHI    1
18  1990-91         Michael Jordan  CHI    1
19  1987-88         Michael Jordan  CHI    1
20  2009-10           LeBron James  CLE    1
21  2008-0

In [125]:
# Joining with the players dataframe
players3 = players2.copy()

players3 = pd.merge(players3, mvps2, how='left', left_on=['name','TEAM','Season'], right_on = ['name','TEAM','Season'])
players3[players3['MVP']==1][['Season','name','TEAM']].sort_values('Season')

,Season,name,TEAM
18798,1955-56,Bob Pettit,STL
5317,1956-57,Bob Cousy,BOS
20726,1957-58,Bill Russell,BOS
18795,1958-59,Bob Pettit,STL
4469,1959-60,Wilt Chamberlain,PHW
20723,1960-61,Bill Russell,BOS
20722,1961-62,Bill Russell,BOS
20721,1962-63,Bill Russell,BOS
20111,1963-64,Oscar Robertson,CIN
20719,1964-65,Bill Russell,BOS


In [126]:
pd.concat([players3[players3['MVP']==1][['Season','name','TEAM','MVP']],mvps2]).drop_duplicates(keep=False)

,Season,name,TEAM,MVP
22,2007-08,Kobe Bryant,LAL,1.0
37,1994-95,David Robinson,SAS,1.0
39,1982-83,Moses Malone,PHI,1.0
40,1980-81,Julius Erving,PHI,1.0
41,1967-68,Wilt Chamberlain,PHI,1.0
42,1966-67,Wilt Chamberlain,PHI,1.0
43,1965-66,Wilt Chamberlain,PHI,1.0
59,1968-69,Wes Unseld,BAL,1.0


In [130]:
players3[(players3['name']=='Kobe Bryant')][['name', 'TEAM','Season']]

,name,TEAM,Season
3229,Kobe Bryant,HOU,1982-83
3230,Kobe Bryant,SDC,1981-82
3231,Kobe Bryant,SDC,1980-81
3232,Kobe Bryant,SDC,1979-80
3233,Kobe Bryant,PHL,1978-79
3234,Kobe Bryant,PHL,1977-78
3235,Kobe Bryant,PHL,1976-77
3236,Kobe Bryant,PHL,1975-76
